<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Wendy", "transactions": [{"transaction-id": 398, "amount": 358}, {"transaction-id": 1224, "amount": 352}, {"transaction-id": 3190, "amount": 387}, {"transaction-id": 4278, "amount": 457}, {"transaction-id": 5528, "amount": 359}, {"transaction-id": 6584, "amount": 332}, {"transaction-id": 7299, "amount": 368}, {"transaction-id": 7492, "amount": 272}, {"transaction-id": 7591, "amount": 491}, {"transaction-id": 8485, "amount": 401}, {"transaction-id": 9609, "amount": 277}, {"transaction-id": 9974, "amount": 321}]}\n',)

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Wendy", "transactions": [{"transaction-id": 398, "amount": 358}, {"transaction-id": 1224, "amount": 352}, {"transaction-id": 3190, "amount": 387}, {"transaction-id": 4278, "amount": 457}, {"transaction-id": 5528, "amount": 359}, {"transaction-id": 6584, "amount": 332}, {"transaction-id": 7299, "amount": 368}, {"transaction-id": 7492, "amount": 272}, {"transaction-id": 7591, "amount": 491}, {"transaction-id": 8485, "amount": 401}, {"transaction-id": 9609, "amount": 277}, {"transaction-id": 9974, "amount": 321}]}\n',
 '{"id": 1, "name": "Laura", "transactions": [{"transaction-id": 178, "amount": -801}]}\n',
 '{"id": 2, "name": "Dan", "transactions": [{"transaction-id": 34, "amount": 322}, {"transaction-id": 185, "amount": 327}, {"transaction-id": 242, "amount": 285}, {"transaction-id": 714, "amount": 264}, {"transaction-id": 859, "amount": 335}, {"transaction-id": 1017, "amount": 342}, {"transaction-id": 1039, "amount": 296}, {"transaction-id": 1041, "amount": 318}, 

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Wendy',
  'transactions': [{'transaction-id': 398, 'amount': 358},
   {'transaction-id': 1224, 'amount': 352},
   {'transaction-id': 3190, 'amount': 387},
   {'transaction-id': 4278, 'amount': 457},
   {'transaction-id': 5528, 'amount': 359},
   {'transaction-id': 6584, 'amount': 332},
   {'transaction-id': 7299, 'amount': 368},
   {'transaction-id': 7492, 'amount': 272},
   {'transaction-id': 7591, 'amount': 491},
   {'transaction-id': 8485, 'amount': 401},
   {'transaction-id': 9609, 'amount': 277},
   {'transaction-id': 9974, 'amount': 321}]},
 {'id': 1,
  'name': 'Laura',
  'transactions': [{'transaction-id': 178, 'amount': -801}]},
 {'id': 2,
  'name': 'Dan',
  'transactions': [{'transaction-id': 34, 'amount': 322},
   {'transaction-id': 185, 'amount': 327},
   {'transaction-id': 242, 'amount': 285},
   {'transaction-id': 714, 'amount': 264},
   {'transaction-id': 859, 'amount': 335},
   {'transaction-id': 1017, 'amount': 342},
   {'transaction-id': 1039, 'am

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 36,
  'name': 'Alice',
  'transactions': [{'transaction-id': 41, 'amount': 510},
   {'transaction-id': 59, 'amount': 465},
   {'transaction-id': 88, 'amount': 528},
   {'transaction-id': 183, 'amount': 518},
   {'transaction-id': 201, 'amount': 459},
   {'transaction-id': 209, 'amount': 539},
   {'transaction-id': 259, 'amount': 524},
   {'transaction-id': 266, 'amount': 494},
   {'transaction-id': 274, 'amount': 506},
   {'transaction-id': 290, 'amount': 476},
   {'transaction-id': 318, 'amount': 492},
   {'transaction-id': 428, 'amount': 526},
   {'transaction-id': 479, 'amount': 509},
   {'transaction-id': 540, 'amount': 526},
   {'transaction-id': 572, 'amount': 511},
   {'transaction-id': 585, 'amount': 510},
   {'transaction-id': 600, 'amount': 509},
   {'transaction-id': 678, 'amount': 510},
   {'transaction-id': 858, 'amount': 489},
   {'transaction-id': 919, 'amount': 506},
   {'transaction-id': 989, 'amount': 501},
   {'transaction-id': 1006, 'amount': 524},
   {'tran

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 191},
 {'name': 'Alice', 'count': 36},
 {'name': 'Alice', 'count': 59},
 {'name': 'Alice', 'count': 78},
 {'name': 'Alice', 'count': 28})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(191, 36, 59, 78, 28)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

61.35777777777778

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 41, 'amount': 510},
  {'transaction-id': 59, 'amount': 465},
  {'transaction-id': 88, 'amount': 528},
  {'transaction-id': 183, 'amount': 518},
  {'transaction-id': 201, 'amount': 459},
  {'transaction-id': 209, 'amount': 539},
  {'transaction-id': 259, 'amount': 524},
  {'transaction-id': 266, 'amount': 494},
  {'transaction-id': 274, 'amount': 506},
  {'transaction-id': 290, 'amount': 476},
  {'transaction-id': 318, 'amount': 492},
  {'transaction-id': 428, 'amount': 526},
  {'transaction-id': 479, 'amount': 509},
  {'transaction-id': 540, 'amount': 526},
  {'transaction-id': 572, 'amount': 511},
  {'transaction-id': 585, 'amount': 510},
  {'transaction-id': 600, 'amount': 509},
  {'transaction-id': 678, 'amount': 510},
  {'transaction-id': 858, 'amount': 489},
  {'transaction-id': 919, 'amount': 506},
  {'transaction-id': 989, 'amount': 501},
  {'transaction-id': 1006, 'amount': 524},
  {'transaction-id': 1170, 'amount': 515},
  {'transaction-id': 1230, 'amount'

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 41, 'amount': 510},
 {'transaction-id': 59, 'amount': 465},
 {'transaction-id': 88, 'amount': 528})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(510, 465, 528)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

835.3929231103546

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 108), ('Alice', 108), ('Alice', 117), ('Alice', 117), ('Bob', 155), ('Bob', 155), ('Bob', 168), ('Bob', 169), ('Charlie', 120), ('Charlie', 120), ('Charlie', 130), ('Charlie', 130), ('Dan', 137), ('Dan', 138), ('Dan', 146), ('Dan', 149), ('Edith', 108), ('Edith', 108), ('Edith', 117), ('Edith', 117), ('Frank', 72), ('Frank', 72), ('Frank', 78), ('Frank', 78), ('George', 95), ('George', 95), ('George', 102), ('George', 103), ('Hannah', 78), ('Hannah', 79), ('Hannah', 157), ('Ingrid', 104), ('Ingrid', 104), ('Ingrid', 192), ('Jerry', 144), ('Jerry', 144), ('Jerry', 153), ('Jerry', 156), ('Kevin', 35), ('Kevin', 36), ('Kevin', 38), ('Kevin', 38), ('Laura', 166), ('Laura', 168), ('Laura', 178), ('Laura', 182), ('Michael', 107), ('Michael', 108), ('Michael', 115), ('Michael', 116), ('Norbert', 94), ('Norbert', 96), ('Norbert', 102), ('Norbert', 103), ('Oliver', 156), ('Oliver', 156), ('Oliver', 169), ('Oliver', 169), ('Patricia', 105), ('Patricia', 107), ('Patricia', 115), ('Patr

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 450), ('Bob', 647), ('Charlie', 500), ('Dan', 570), ('Edith', 450), ('Frank', 300), ('George', 395), ('Hannah', 314), ('Ingrid', 400), ('Jerry', 597), ('Kevin', 147), ('Laura', 694), ('Michael', 446), ('Norbert', 395), ('Oliver', 650), ('Patricia', 443), ('Quinn', 717), ('Ray', 545), ('Sarah', 435), ('Tim', 300), ('Ursula', 499), ('Victor', 399), ('Wendy', 488), ('Xavier', 306), ('Yvonne', 714), ('Zelda', 447)]
CPU times: user 157 ms, sys: 6.54 ms, total: 164 ms
Wall time: 618 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Wendy,"[{'transaction-id': 398, 'amount': 358}, {'tra..."
1,1,Laura,"[{'transaction-id': 178, 'amount': -801}]"
2,2,Dan,"[{'transaction-id': 34, 'amount': 322}, {'tran..."
3,3,Sarah,"[{'transaction-id': 315, 'amount': 170}, {'tra..."
4,4,Michael,"[{'transaction-id': 43, 'amount': 493}, {'tran..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 212 ms, sys: 30 ms, total: 242 ms
Wall time: 1.8 s


name
Alice      450
Bob        647
Charlie    500
Dan        570
Edith      450
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Wendy', 'amount': 358, 'transaction-id': 398},
 {'id': 0, 'name': 'Wendy', 'amount': 352, 'transaction-id': 1224},
 {'id': 0, 'name': 'Wendy', 'amount': 387, 'transaction-id': 3190})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Wendy,358,398
1,0,Wendy,352,1224
2,0,Wendy,387,3190
3,0,Wendy,457,4278
4,0,Wendy,359,5528


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 216 ms, sys: 15.4 ms, total: 231 ms
Wall time: 1.52 s


name
Alice       27611
Bob         35155
Charlie     22428
Dan         29267
Edith       12102
Frank       16760
George      10596
Hannah      12391
Ingrid      21238
Jerry       35879
Kevin        3735
Laura       29613
Michael     17700
Norbert     16357
Oliver      30444
Patricia     9675
Quinn       18801
Ray         15502
Sarah       18659
Tim         15555
Ursula      14860
Victor      13293
Wendy       14041
Xavier      13891
Yvonne      31469
Zelda       12978
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()